In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from scipy.stats import norm

In [19]:
# ebd_left_CINGULATE = pd.read_csv(/ccc/scratch/cont003/n4h00001/dufourna/Left_PCS/11-25-46_0/ukb_random_epoch80_embeddings/left_CINGULATE.csv')
ebd_left_CINGULATE = pd.read_csv('/neurospin/dico/adufournet/mycode/Output/2024-07-22/11-25-46_0/ukb_random_epoch80_embeddings/full_embeddings.csv')
# covar_file = pd.read_csv(/ccc/scratch/cont003/n4h00001/dufourna/UkBioBank/participants.csv')
covar_file = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/participants.csv')
covar_file = covar_file[["participant_id", "BrainVolumeFreeSurfer_mm3"]]

In [20]:
#ebd_left_CINGULATE['IID'] = ebd_left_CINGULATE['ID'].apply(lambda x: x[-7:]).astype(int)
#ebd_left_CINGULATE = ebd_left_CINGULATE.drop('ID', axis=1)
#ebd_left_CINGULATE.to_csv("~/Bureau/Irene/2024-07-22/11-25-46_0/ukb_random_epoch80_embeddings/full_embeddings.csv", index=False)

In [ ]:
pheno = pd.read_csv("ccc/scratch/cont003/n4h00001/dufourna/Left_PCS/11-25-46_0/ukb_random_epoch80_embeddings/left_CINGULATE_pheno.phe", sep='\t')
cov = pd.read_csv("ccc/scratch/cont003/n4h00001/dufourna/Left_PCS/11-25-46_0/ukb_random_epoch80_embeddings/left_CINGULATE_covar.cov", sep='\t')
pheno_cov = pd.merge(left=pheno, right=cov.drop('#FID', axis=1), on='IID', how='inner')

pheno_cov = pd.merge(left=pheno_cov, right=covar_file, how='inner', left_on='IID', right_on='participant_id')
pheno_cov = pheno_cov.drop("participant_id", axis=1)

In [ ]:
# Define the covariates
list_cov = ['Age','Cheadle','Sex','PC01','PC02','PC03','PC04','PC05','PC06','PC07','PC08','PC09','PC10','BrainVolumeFreeSurfer_mm3']

# List of dimensions (that can be seen as phenotypes)
phenotype_col = [f'dim{i}' for i in range(1,257)]

for col in phenotype_col:
    # Fit the linear model
    X = pheno_cov[list_cov]
    X = sm.add_constant(X)  # Adds a constant term to the predictor
    y = pheno_cov[col]
    model = sm.OLS(y, X, missing='drop').fit()

    # Get the intercept and residuals
    intercept = model.params.loc['const']
    residuals = model.resid

    # Update the phenotype column with the pre-residualized values
    pheno_cov[col] = intercept + residuals

In [ ]:
for dim_i in phenotype_col:
    # Calculate the ECDF values for 'dim_i'
    ecdf_values = pheno_cov[dim_i].rank(method='average') / len(pheno_cov[dim_i])

    # Apply the quantile function of the normal distribution
    qnorm_values = norm.ppf(ecdf_values - 0.5 / len(pheno_cov[dim_i]))

    # Assign the transformed values back to 'dim_i'
    pheno_cov[dim_i] = qnorm_values

In [ ]:
pheno_cov[phenotype_col].to_csv("ccc/scratch/cont003/n4h00001/dufourna/Left_PCS/11-25-46_0/ukb_random_epoch80_embeddings/left_CINGULATE_pheno.txt", sep='\t', index=False)